In [1]:
using Pkg, Revise
Pkg.activate(".")

  Activating environment at `~/Desktop/SelectivelyAdaptiveLasso/Project.toml`


In [2]:
import SelectivelyAdaptiveLasso as SAL

┌ Info: Precompiling SelectivelyAdaptiveLasso [52f33b2b-83b2-4cfe-8aa4-4763af86ef4f]
└ @ Base loading.jl:1342


In [44]:
import Statistics: mean
import Distributions as Dist
import LinearAlgebra: diagm

n, p = 5000, 10

X_dist = Dist.reshape(
    Dist.MvNormal(zeros(p), 0.1ones(p,p) + 0.9diagm(ones(p))), 
    (1,p)
)

X = vcat(rand(X_dist, n)...);
# Y = rand(n)
Y = 5X[:,1] + X[:,2].^2 - 3X[:,3].*X[:,4]
Y = Y .- mean(Y)

n_val = Int(round(n/2))
X_val, Y_val = X[1:n_val,:], Y[1:n_val]
X, Y = X[(n_val+1):end,:], Y[(n_val+1):end];

In [97]:
sal_spec = SAL.SALSpec(
    max_iter=5000,
    λ_max = 0.0001,
    m_subsample=400,
    n_subsample=400
)
@time sal_fit, log = SAL.fit(
    sal_spec, 
    X, Y, 
    X_val=X_val, Y_val=Y_val
)
loss, loss_val, _, _ = log;

(33.47313505813793, 36.413559308651095, 0.0001, Set([0, 3]))
(6.711836476339974, 8.426420961435483, 0.0001, Set([0, 8]))
(5.111477029137681, 6.888365982375967, 0.0001, Set([0, 4, 1]))
(3.2249924334910736, 4.308230088656993, 0.0001, Set([0, 4]))
(2.4209828909549995, 3.525945306900189, 0.0001, Set([0, 5]))
(1.8845229269095602, 2.9094939485633264, 0.0001, Set([0, 8]))
(1.6010909289841977, 2.485466413756345, 0.0001, Set([0, 8]))
(1.557354307736627, 2.466784989096666, 0.0001, Set([0, 5, 3]))
(1.512374267229436, 2.4344891106659654, 0.0001, Set([0, 4]))
(1.4805888009901047, 2.429170793551061, 0.0001, Set([0, 9]))
(1.3389143310175422, 2.218616692919364, 0.0001, Set([0, 1]))
(1.2775718174605237, 2.1469488607009906, 0.0001, Set([0, 4, 7, 1]))
(1.2480229818073965, 2.1453065116913526, 0.0001, Set([0, 2]))
(1.1503259686163176, 1.948095717797766, 0.0001, Set([0, 2, 1]))
(1.1417443447432147, 1.9403210560268693, 0.0001, Set([0, 2]))
(1.1309439230199307, 1.9274412171388933, 0.0001, Set([0, 2, 1]))
(1.1

In [98]:
sal_fit.β

Dict{Set{CartesianIndex{2}}, Float64} with 613 entries:
  Set([CartesianIndex(0, 0), CartesianIndex(14, 4), CartesianInd… => 0.125395
  Set([CartesianIndex(0, 0), CartesianIndex(80, 1)])              => 0.0338985
  Set([CartesianIndex(0, 0), CartesianIndex(283, 3), CartesianIn… => 0.006145
  Set([CartesianIndex(0, 0), CartesianIndex(2, 1), CartesianInde… => 0.0832929
  Set([CartesianIndex(0, 0), CartesianIndex(326, 3)])             => 0.119153
  Set([CartesianIndex(0, 0), CartesianIndex(371, 4)])             => 0.158711
  Set([CartesianIndex(0, 0), CartesianIndex(285, 1)])             => 0.647757
  Set([CartesianIndex(0, 0), CartesianIndex(46, 3), CartesianInd… => 0.139582
  Set([CartesianIndex(0, 0), CartesianIndex(300, 1)])             => 0.138083
  Set([CartesianIndex(0, 0), CartesianIndex(399, 2)])             => -0.0930372
  Set([CartesianIndex(0, 0), CartesianIndex(2, 10), CartesianInd… => 0.0821575
  Set([CartesianIndex(0, 0), CartesianIndex(80, 3)])              => 1.17999
  Se

In [99]:
Ŷ = SAL.predict(sal_fit, X_val)
mean((Y_val - Ŷ).^2)

0.97951709181406

In [90]:
using XGBoost

In [102]:
bst = xgboost(X, 800, label = Y, eta = 0.1, max_depth = 4)
Ŷ = predict(bst, X_val)

[1]	train-rmse:5.417451
[2]	train-rmse:5.054398
[3]	train-rmse:4.737821
[4]	train-rmse:4.461241
[5]	train-rmse:4.217824
[6]	train-rmse:4.008256
[7]	train-rmse:3.827026
[8]	train-rmse:3.669997
[9]	train-rmse:3.532923
[10]	train-rmse:3.412463
[11]	train-rmse:3.300926
[12]	train-rmse:3.209804
[13]	train-rmse:3.124469
[14]	train-rmse:3.041675
[15]	train-rmse:2.980562
[16]	train-rmse:2.926010
[17]	train-rmse:2.877160
[18]	train-rmse:2.826645
[19]	train-rmse:2.789753
[20]	train-rmse:2.757839
[21]	train-rmse:2.708048
[22]	train-rmse:2.682953
[23]	train-rmse:2.658247
[24]	train-rmse:2.629360
[25]	train-rmse:2.608744
[26]	train-rmse:2.588466
[27]	train-rmse:2.549759
[28]	train-rmse:2.533902
[29]	train-rmse:2.420035
[30]	train-rmse:2.392046
[31]	train-rmse:2.373642
[32]	train-rmse:2.357354
[33]	train-rmse:2.314481
[34]	train-rmse:2.306185
[35]	train-rmse:2.289238
[36]	train-rmse:2.279944
[37]	train-rmse:2.249596
[38]	train-rmse:2.230621
[39]	train-rmse:2.221936
[40]	train-rmse:2.210713
[41]	trai

2500-element Vector{Float32}:
  0.8695484
 -4.3781652
  5.1574373
 -1.3740623
  2.907614
  0.973489
  1.9252954
  9.204386
  2.6348677
 -3.2637317
 -7.1911488
 -3.0417995
  1.5024859
  ⋮
 -1.4519619
 -2.6781805
  0.9302507
  2.9939332
 -7.5479956
 -1.7643231
  0.35752773
 -9.310436
  3.893793
 -0.80053425
  2.4157712
  5.3253446

In [103]:
mean((Y_val - Ŷ).^2)

1.3502629050840165

In [16]:
BasisIndex = Set{Tuple{Int, Int}}

Set{Tuple{Int64, Int64}}

In [18]:
BasisIndex([(0,0)])

Set{Tuple{Int64, Int64}} with 1 element:
  (0, 0)

In [20]:
?CartesianIndex

search: CartesianIndex CartesianIndices



```
CartesianIndex(i, j, k...)   -> I
CartesianIndex((i, j, k...)) -> I
```

Create a multidimensional index `I`, which can be used for indexing a multidimensional array `A`.  In particular, `A[I]` is equivalent to `A[i,j,k...]`.  One can freely mix integer and `CartesianIndex` indices; for example, `A[Ipre, i, Ipost]` (where `Ipre` and `Ipost` are `CartesianIndex` indices and `i` is an `Int`) can be a useful expression when writing algorithms that work along a single dimension of an array of arbitrary dimensionality.

A `CartesianIndex` is sometimes produced by [`eachindex`](@ref), and always when iterating with an explicit [`CartesianIndices`](@ref).

# Examples

```jldoctest
julia> A = reshape(Vector(1:16), (2, 2, 2, 2))
2×2×2×2 Array{Int64, 4}:
[:, :, 1, 1] =
 1  3
 2  4

[:, :, 2, 1] =
 5  7
 6  8

[:, :, 1, 2] =
  9  11
 10  12

[:, :, 2, 2] =
 13  15
 14  16

julia> A[CartesianIndex((1, 1, 1, 1))]
1

julia> A[CartesianIndex((1, 1, 1, 2))]
9

julia> A[CartesianIndex((1, 1, 2, 1))]
5
```


In [22]:
derpidx = Set([CartesianIndex(2,1), CartesianIndex(1,2)])

Set{CartesianIndex{2}} with 2 elements:
  CartesianIndex(2, 1)
  CartesianIndex(1, 2)

In [30]:
A = rand(10,2,2)

10×2×2 Array{Float64, 3}:
[:, :, 1] =
 0.593408  0.820085
 0.84067   0.618129
 0.592314  0.118786
 0.340206  0.155653
 0.973213  0.769094
 0.829289  0.735042
 0.685964  0.535363
 0.100297  0.228542
 0.431073  0.93619
 0.693552  0.127805

[:, :, 2] =
 0.119806   0.619967
 0.353965   0.225389
 0.888493   0.946976
 0.9949     0.989663
 0.19066    0.680088
 0.0793931  0.495023
 0.177553   0.76208
 0.884334   0.860417
 0.789164   0.762087
 0.230166   0.752132

In [33]:
prod(A[:,[derpidx...]], dims=2)[:,1]

10-element Vector{Float64}:
 0.09825128482140733
 0.21879612334162674
 0.10554059350350542
 0.1548587356342069
 0.14663578004130753
 0.05835721670246334
 0.09505540576799612
 0.2021077208753243
 0.7388067823837791
 0.02941631891154574